## Decorator
A decorator is a powerful and elegant way to modify or enhance functions and methods without directly changing their code.  

Decorators use the `@` symbol followed by the decorator function name, placed directly above the function you want to modify.


**What a Decorator Does:**

1. **Takes a function as input:**  A decorator is itself a function that accepts another function as an argument.

2. **Wraps the function:** It creates a new function (often called a "wrapper" function) that encapsulates the original function.

3. **Adds functionality:** The wrapper function can execute code *before* calling the original function, *after* calling the original function, or *both*. This is where the modification or enhancement happens.

4. **Returns the modified function:** The decorator returns the wrapper function, effectively replacing the original function with the enhanced version.

5.  **`functools.wraps`**: This is a crucial addition. When you use a decorator, the decorated function (`greet` in your example) loses its original metadata, such as its name, docstring, and argument annotations. This can cause issues with debugging and other tools. The `@functools.wraps(func)` decorator copies this metadata from the original function to the new wrapper function.

6.  **Type Hinting**: Adding type hints to the decorator's arguments and return values can improve code readability and help with static analysis.

Here's the enhanced version:

```python
import functools
from typing import Callable, Any

def decorator(func: Callable) -> Callable:
    @functools.wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        print("before")
        val = func(*args, **kwargs)
        print("after")
        return val
    return wrapper

@decorator
def greet(name: str) -> str:
    """This function greets a person."""
    return f"welcome {name}"

# Example of how to use the function
print(greet("Alice"))

# Now, the function's metadata is preserved
print(f"Function name: {greet.__name__}")
print(f"Function docstring: {greet.__doc__}")
```

In this revised code, `functools.wraps` ensures that when you call `greet.__name__` or `greet.__doc__`, you get the correct information from the original function, not the `wrapper` function created by the decorator. This is considered a **best practice** for writing decorators in Python.
